In [19]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import os
import anndata
import scipy as sp
import umap.umap_ as umap
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

MOSTA

In [ ]:
sample=['E9.5_E1S1', 'E10.5_E2S1', 'E11.5_E1S1', 'E12.5_E1S1']
Batch_list = []
for i in np.arange(len(sample)):
    adata=sc.read_h5ad("/data02/tguo/space_batch_effect_data/MOSTA/origin/"+sample[i]+".MOSTA.h5ad")
    adata.X = adata.layers['count']
    adata.obs_names = [x + '_' + sample[i] for x in adata.obs_names]
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=5000)
    adata = adata[:, adata.var['highly_variable']]
    Batch_list.append(adata)
    
adata_concat = anndata.concat(Batch_list, label="slice_name", keys=sample)
vf=np.array(adata_concat.var.index)
for i in np.arange(len(sample)):
    adata1=adata_concat[adata_concat.obs['slice_name']==sample[i],:]
    cells=adata1.obs_names
    mat=pd.DataFrame(adata1.X.todense(),index=cells,columns=vf)
    coord=pd.DataFrame(adata1.obsm['spatial'],index=cells,columns=['x','y'])
    meta=adata1.obs[['annotation','slice_name']]
    meta.columns=['celltype','batch']
    meta.index=cells
    mat.to_csv("/data02/tguo/space_batch_effect_data/MOSTA/gtt_input/"+sample[i]+"_mat.csv")
    meta.to_csv("/data02/tguo/space_batch_effect_data/MOSTA/gtt_input/"+sample[i]+"_meta.csv")
    coord.to_csv("/data02/tguo/space_batch_effect_data/MOSTA/gtt_input/"+sample[i]+"_coord.csv")

DLPFC

Donor1/Donor2/Donor3: union of 3000 HVG from each samples;


All Donor: union of 1000 HVG from each samples;

In [26]:
dirs="/data02/tguo/space_batch_effect/human_DLPFC_10x/"
sample_name=[151507,151508,151509,151510,151669,151670,151671,151672,151673,151674,151675,151676]
IDX=np.arange(0,12)
VF=[]
MAT=[]
flags=str(sample_name[IDX[0]])
for i in np.arange(1,len(IDX)):
    flags=flags+'-'+str(sample_name[IDX[i]])
flags=flags+"_"
for k in np.arange(len(IDX)):
    adata = sc.read_visium(path=dirs+"scanpy_file/"+str(sample_name[IDX[k]])+"/",
                       count_file="filtered_feature_bc_matrix.h5")
    adata.var_names_make_unique()
    Ann_df=pd.read_csv(dirs+"input/"+str(sample_name[IDX[k]])+"_label.txt", sep=',', header=0, index_col=0)
    Ann_df.columns=['Ground Truth','Sample']
    Ann_df.index=[i.split('-')[1]+'-'+i.split('-')[2] for i in Ann_df.index]
    adata=adata[Ann_df.index,:]
    adata.obs['Ground Truth']=Ann_df.loc[:,'Ground Truth']
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=1000)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.obs['batch']=str(sample_name[IDX[k]])
    cells=[str(sample_name[IDX[k]])+'-'+i for i in adata.obs_names]
    mat1=pd.DataFrame(adata.X.toarray(),columns=adata.var_names,index=cells)
    coord1=pd.DataFrame(adata.obsm['spatial'],columns=['x','y'],index=cells)
    meta1=adata.obs[['Ground Truth', 'batch']]
    meta1.columns=['celltype','batch']
    meta1.index=cells
    meta1.to_csv(dirs+"gtt_input_scanpy/"+flags+str(sample_name[IDX[k]])+"_label-1.txt")
    coord1.to_csv(dirs+"gtt_input_scanpy/"+flags+str(sample_name[IDX[k]])+"_positions-1.txt")
    MAT.append(mat1)
    VF=np.union1d(VF,adata.var_names[adata.var['highly_variable']])

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call 

In [28]:
for i in np.arange(len(IDX)):
    mat=MAT[i]
    mat=mat.loc[:,VF]
    mat.to_csv(dirs+"gtt_input_scanpy/"+flags+str(sample_name[IDX[i]])+"_features-1.txt")